In [19]:
# combining data

import os
import json 
TEST_RATIO = 0.2
TRAIN_RATIO = 0.8

# directory of the donwnloaded data
dir = r"C:\Users\Emma\Downloads\school\Big_Data\project\amazon_review_data"
# directory for the combined files to write to
combined_test = r"C:\Users\Emma\Downloads\school\Big_Data\project\big-data-final-project\combined_test.json"
combined_train = r"C:\Users\Emma\Downloads\school\Big_Data\project\big-data-final-project\combined_train.json"

def split_train_test(filename):
    with open(filename, 'r') as f:

        # count how many lines to read
        content = f.readlines()
        lines_total = len(content)
        lines_test = int(lines_total*TEST_RATIO)
        lines_train = int(lines_total*TRAIN_RATIO)
        #print(lines_total, lines_test, lines_train)

        # write the first 20% into testing json
        with open(combined_test, 'a') as output_file:
            for i in range(lines_test):
                line = content[i]
                line.strip()
                data = json.loads(line)
                json.dump(data, output_file, indent=4)
                output_file.write('\n')
        output_file.close()

        # write the rest into training json
        with open(combined_train, 'a') as output_file:
            for i in range(lines_test,lines_test+lines_train):
                line = content[i]
                line.strip()
                data = json.loads(line)
                json.dump(data, output_file, indent=4)
                output_file.write('\n')
        output_file.close()
    

def combine_data():
    for root, _, files in os.walk(dir):
        for filename in files:
            print(filename)
            split_train_test(os.path.join(root, filename))        

In [20]:
combine_data()

All_Beauty_5.json
AMAZON_FASHION_5.json
Appliances_5.json
Arts_Crafts_and_Sewing_5.json
Automotive_5.json
CDs_and_Vinyl_5.json
Cell_Phones_and_Accessories_5.json
Digital_Music_5.json
Electronics_5.json
Gift_Cards_5.json
Grocery_and_Gourmet_Food_5.json
Home_and_Kitchen_5.json
Industrial_and_Scientific_5.json
Kindle_Store_5.json
Luxury_Beauty_5.json
Magazine_Subscriptions_5.json
Movies_and_TV_5.json
Musical_Instruments_5.json
Office_Products_5.json
Patio_Lawn_and_Garden_5.json
Pet_Supplies_5.json
Prime_Pantry_5.json
Software_5.json
Sports_and_Outdoors_5.json
Toys_and_Games_5.json
Video_Games_5.json


In [1]:
# did not shuffle the data
# google drive link to combined data
# https://drive.google.com/file/d/12ZjwXcF8WiZbUCeSzNkBnXKMUzmsqPF0/view?usp=drive_link

In [ ]:
# divider------

In [8]:
import findspark
findspark.init()

In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, FloatType, BooleanType, IntegerType
from pyspark.sql.functions import col, struct
from pyspark.sql.functions import when

# create a SparkSession
spark = SparkSession.builder.appName("ReadJSON").getOrCreate()

In [10]:
# define schema
schema = StructType(
    [
        StructField("asin", StringType(), True),
        StructField("reviewerID", StringType(), True),
        StructField("vote", StringType(), True),
        StructField("overall", StringType(), True),
        StructField("summary", StringType(), True),
        StructField("reviewText", StringType(), True)
    ]
)

# read a JSON file into a DataFrame
df = spark.read.schema(schema).json(r"C:\Users\Emma\Downloads\school\Big_Data\project\amazon_review_data\AMAZON_FASHION_5.json")
df.count()

3176

In [43]:
# sort by vote
df = df.withColumn('vote', col('vote').cast(IntegerType()))
df_byVote = df.orderBy(df["vote"].desc())
df_byVote.show()

+----------+--------------+----+-------+--------------------+--------------------+
|      asin|    reviewerID|vote|overall|             summary|          reviewText|
+----------+--------------+----+-------+--------------------+--------------------+
|B000YFSR4W|A3GJ3DJU1RXOHN|  35|    4.0|       size runs big|great price for t...|
|B000YFSR5G|A3GJ3DJU1RXOHN|  35|    4.0|       size runs big|great price for t...|
|B000YFSR4W|A3GJ3DJU1RXOHN|  35|    4.0|       size runs big|great price for t...|
|B000YFSR5G|A3GJ3DJU1RXOHN|  35|    4.0|       size runs big|great price for t...|
|B001LNSY2Q|A3GJ3DJU1RXOHN|  35|    4.0|       size runs big|great price for t...|
|B0014F7B98| ATS6HOUGOVX0D|  22|    5.0|Exactly What I Wa...|Tried them on in ...|
|B009MA34NY| ATS6HOUGOVX0D|  22|    5.0|Exactly What I Wa...|Tried them on in ...|
|B014IBJKNO| A8DR2Z71DERN0|  22|    1.0|Poor quality for ...|After less than a...|
|B014IBJKNO| ATS6HOUGOVX0D|  22|    5.0|Exactly What I Wa...|Tried them on in ...|
|B00

In [38]:
# rate reviews with overall score
# 5,4 as good/ 1,2,3 as bad

df_withRank = df.withColumn("rank_onOverall",
                            when(df["overall"] > 3, "good")
                             .otherwise("bad"))
df_withRank.show()

+----------+--------------+----+-------+--------------------+--------------------+--------------+
|      asin|    reviewerID|vote|overall|             summary|          reviewText|rank_onOverall|
+----------+--------------+----+-------+--------------------+--------------------+--------------+
|B000K2PJ4K| ALJ66O1Y6SLHA|NULL|    5.0|          Five Stars|Great product and...|          good|
|B000K2PJ4K| ALJ66O1Y6SLHA|NULL|    5.0|          Five Stars|Great product and...|          good|
|B000K2PJ4K| ALJ66O1Y6SLHA|NULL|    5.0|          Five Stars|Great product and...|          good|
|B000K2PJ4K| ALJ66O1Y6SLHA|NULL|    5.0|          Five Stars|Great product and...|          good|
|B000K2PJ4K| ALJ66O1Y6SLHA|NULL|    5.0|          Five Stars|Great product and...|          good|
|B000K2PJ4K|A3W11493KS6Z2L|NULL|    3.0|               Oops!|Waaay too small. ...|           bad|
|B000K2PJ4K|A3W11493KS6Z2L|NULL|    5.0|               Great|Stays vibrant aft...|          good|
|B000K2PJ4K|A3W11493

In [ ]:
# nltk pretrained sentiment analyzer

In [19]:
# tmp

from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

In [25]:
sia.polarity_scores("These insoles help my heel feels much better. I would never wear a pair of athletic ...")

{'neg': 0.0, 'neu': 0.699, 'pos': 0.301, 'compound': 0.6808}

In [44]:
sia.polarity_scores("Nice pants but too small for a medium. Return it")

{'neg': 0.0, 'neu': 0.808, 'pos': 0.192, 'compound': 0.2263}